In [25]:
import pandas as pd
import numpy as np 
import os

from tqdm import tqdm
from string import digits

In [26]:
path = '/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Fishbase_HTML/HTML_files/'

fishbase_df = []

count=0

for file in tqdm(os.listdir(path)):
    fish_name = file.split('.')[0].lower().replace(' ', '_').translate(str.maketrans('','','1234567890'))
    df = pd.read_html(f'{path}{file}')[0]
    df.columns = df.iloc[2,:]
    df.drop([0, 1, 2], inplace=True)
    df['Name used'] = fish_name
    fishbase_df.append(df)

100%|██████████| 69/69 [02:32<00:00,  2.21s/it]


In [27]:
fishbase_df = pd.concat(fishbase_df)
print(len(fishbase_df))
display(fishbase_df.sample(5))

257913


2,Name used,Year,Latitude,Longitude,Catalog No.,Information
5888,alosa_pseudoharengus,1975,41.52,-68.37,8072.ALEWIF,Portal: OBIS. Source: NMFS-NEFSCAlosa pseudoha...
8111,pimephales_notatus,2000,45.95,92.56,37039,St. Croix at St. Croix State Park Portal: GBIF...
12654,lota_lota,NaN,65.17,18.25,723210-161430-19960904-LOTALOTA,"Mattjokkbäcken, Mattjokkbäcken Portal: GBIF. S..."
3602,lepomis_gibbosus,1959,45.72,-76.95,CMNFI 1959-0349.3,Portal: GBIF. Source: CMNData from GBIF data i...
851,lepisosteus_osseus,1999,42.50,-82.46,CID 100251 R,Shut-In Bay (South Pond on NTS map); NE shore ...


In [28]:
fishbase_df.dropna(subset=['Year', 'Latitude', 'Longitude'], inplace=True)

fishbase_df['lat_long'] = (fishbase_df['Latitude'].astype(float).round(3).astype(str) + '_' + 
                           fishbase_df['Longitude'].astype(float).round(3).astype(str))
fishbase_df.drop(columns=['Catalog No.', 'Information'], inplace=True)

print(len(fishbase_df))
display(fishbase_df.sample(5))

193682


2,Name used,Year,Latitude,Longitude,lat_long
14559,salmo_trutta,1995,61.40,15.79,61.4_15.79
4782,lepomis_cyanellus,1964,36.50,-78.50,36.5_-78.5
3815,oncorhynchus_mykiss,2004,50.68,-127.35,50.68_-127.35
7,noturus_gyrinus,1984,49.62,-100.25,49.62_-100.25
29167,oncorhynchus_tshawytscha,2007,45.69,-120.78,45.69_-120.78


In [30]:
%%time
for col in tqdm(fishbase_df['Name used'].unique()):
    fishbase_df[col] = np.where(fishbase_df.isin([col]).any(1), 1, np.nan)
    fishbase_df[col] = fishbase_df[col].fillna(fishbase_df.groupby(['Year', 'lat_long'])[col].transform('mean'))

display(fishbase_df.sample(5))  
print(fishbase_df['Year'].min(), fishbase_df['Year'].max())

100%|██████████| 50/50 [01:15<00:00,  1.51s/it]


2,Name used,Year,Latitude,Longitude,lat_long,alosa_pseudoharengus,ameiurus_nebulosus,amia_calva,aplodinotus_grunniens,campostoma_anomalum,...,pimephales_promelas,pomoxis_nigromaculatus,prosopium_cylindraceum,pungitius_pungitius,salmo_trutta,salvelinus_namaycush,salvelinus_alpinus,salvelinus_confluentus,salvelinus_fontinalis,thymallus_arcticus
354,oncorhynchus_nerka,1961,60.54,-145.67,60.54_-145.67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3083,alosa_pseudoharengus,1992,42.73,-70.52,42.73_-70.52,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3978,lota_lota,1992,60.29,13.18,60.29_13.18,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2898,hypentelium_nigricans,1998,43.41,-79.74,43.41_-79.74,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2229,salmo_trutta,1993,58.96,12.22,58.96_12.22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


1003 9171
CPU times: user 39 s, sys: 33.5 s, total: 1min 12s
Wall time: 1min 15s


In [37]:
fishbase_df = fishbase_df[~fishbase_df['Year'].isin(['1003', '2994', '9171'])]

In [39]:
print(len(fishbase_df))
fishbase_df.drop(columns=['Name used'], inplace=True)
fishbase_df.drop_duplicates(inplace=True)
print(len(fishbase_df))

display(fishbase_df.sample(5))
fishbase_df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Fishbase_HTML/Processed/fishbase_1800_2009_occurence.csv', 
                   index=False)

193679
60315


/home/arshad/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/arshad/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2,Year,Latitude,Longitude,lat_long,alosa_pseudoharengus,ameiurus_nebulosus,amia_calva,aplodinotus_grunniens,campostoma_anomalum,carassius_auratus,...,pimephales_promelas,pomoxis_nigromaculatus,prosopium_cylindraceum,pungitius_pungitius,salmo_trutta,salvelinus_namaycush,salvelinus_alpinus,salvelinus_confluentus,salvelinus_fontinalis,thymallus_arcticus
2089,1988,45.27,5.94,45.27_5.94,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
790,2003,42.68,-82.26,42.68_-82.26,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5359,1984,59.74,12.88,59.74_12.88,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
7829,1973,45.03,-74.98,45.03_-74.98,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2260,2002,43.05,-81.30,43.05_-81.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
fishbase_df['Year'].max()

'9171'